In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam

2023-09-22 02:46:10.863285: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 02:46:11.495923: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:
2023-09-22 02:46:11.496001: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/

In [2]:
df = pd.read_csv('../Data/CleanedData.csv')

In [3]:
texts = df['transformed text'].astype(str).tolist()
labels = df['subject'].astype(int).tolist()

In [4]:
max_sequence_length = 1000  # Set your desired sequence length
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_sequence_length)
y = tf.keras.utils.to_categorical(labels, num_classes=6)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

def lr_scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)



initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


    

In [14]:
lr_schedule = LearningRateScheduler(lr_scheduler)

# Train the model
history = model.fit(
    X_train, y_train, 
    epochs=30, batch_size=64, validation_split=0.2, 
    callbacks=[early_stopping, lr_schedule]
)

Epoch 1/30
432/432 [==============================] - 29s 65ms/step - loss: 1.0258 - accuracy: 0.6077 - val_loss: 0.6251 - val_accuracy: 0.7593 - lr: 0.0010
Epoch 2/30
432/432 [==============================] - 19s 44ms/step - loss: 0.5044 - accuracy: 0.8062 - val_loss: 0.5578 - val_accuracy: 0.7738 - lr: 0.0010
Epoch 3/30
432/432 [==============================] - 13s 30ms/step - loss: 0.3569 - accuracy: 0.8435 - val_loss: 0.6393 - val_accuracy: 0.7690 - lr: 0.0010
Epoch 4/30
432/432 [==============================] - 12s 27ms/step - loss: 0.3032 - accuracy: 0.8594 - val_loss: 0.7366 - val_accuracy: 0.7719 - lr: 0.0010
Epoch 5/30
432/432 [==============================] - 10s 23ms/step - loss: 0.2636 - accuracy: 0.8736 - val_loss: 0.8950 - val_accuracy: 0.7659 - lr: 0.0010
Epoch 6/30
432/432 [==============================] - 10s 22ms/step - loss: 0.2358 - accuracy: 0.8874 - val_loss: 0.9262 - val_accuracy: 0.7197 - lr: 9.0484e-04
Epoch 7/30
432/432 [==============================] - 

In [15]:
y_pred = model.predict(X_test)

# Convert predictions to class labels
predicted_labels = np.argmax(y_pred, axis=1)

# Calculate F1 score
test_f1_score = f1_score(np.argmax(y_test, axis=1), predicted_labels, average='weighted')

print("Test F1 Score:", test_f1_score)

270/270 [==============================] - 0s 2ms/step
Test F1 Score: 0.7490039669400765
